In [1]:
# import data from the csv
import pandas as pd
import numpy as py
import requests
import json
from config import tmdb_api_key

csv_file = "netflix_data_clean.csv"
netflix_file_clean_df = pd.read_csv(csv_file)
netflix_file_clean_df.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...
1,20,80060297,Movie,Manhattan Romance,Tom O'Brien,"Tom O'Brien, Katherine Waterston, Caitlin Fitz...",United States,"September 8, 2017",2014,TV-14,98 min,"Comedies, Independent Movies, Romantic Movies",A filmmaker working on a documentary about lov...
2,23,70304988,Movie,Stonehearst Asylum,Brad Anderson,"Kate Beckinsale, Jim Sturgess, David Thewlis, ...",United States,"September 8, 2017",2014,PG-13,113 min,"Horror Movies, Thrillers","In 1899, a young doctor arrives at an asylum f..."
3,24,80057700,Movie,The Runner,Austin Stark,"Nicolas Cage, Sarah Paulson, Connie Nielsen, W...",United States,"September 8, 2017",2015,R,90 min,"Dramas, Independent Movies",A New Orleans politician finds his idealistic ...
4,25,80045922,Movie,6 Years,Hannah Fidell,"Taissa Farmiga, Ben Rosenfield, Lindsay Burdge...",United States,"September 8, 2015",2015,NR,80 min,"Dramas, Independent Movies, Romantic Movies",As a volatile young couple who have been toget...


In [2]:
# Grab a small subset of data for testing
test_data = netflix_file_clean_df.loc[netflix_file_clean_df["director"] == "David Fincher"]
test_data.head()

,Unnamed: 0,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
291,1196,70044686,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
374,1416,1178866,Movie,The Game,David Fincher,"Michael Douglas, Sean Penn, Deborah Kara Unger...",United States,"November 1, 2019",1997,R,129 min,Thrillers,Nicholas receives an unusual birthday present ...
1200,5069,60022700,Movie,Panic Room,David Fincher,"Jodie Foster, Forest Whitaker, Dwight Yoakam, ...",United States,"August 1, 2019",2002,R,112 min,Thrillers,A woman and her daughter are caught in a game ...


In [3]:
url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query="
titles = netflix_file_clean_df["title"]
movie_name = []
original_title = []
popularity = []
vote_count = []
vote_average = []
poster_path = []

for title in titles:
    title = title.replace(" ", "+")
    response = requests.get(url + title)
    response_json = response.json()
    #print(response)
        
    try:
           movie_name.append(response_json['results'][0]['title'])
    except:
           movie_name.append('na')
        
    try:
           original_title.append(response_json['results'][0]['original_title'])
    except:
           original_title.append('na')
        
    try:
           popularity.append(response_json['results'][0]['popularity'])
    except:
           popularity.append('na')
       
    try:
           vote_count.append(response_json['results'][0]['vote_count'])
    except:
           vote_count.append('na')

    try:
           vote_average.append(response_json['results'][0]['vote_average'])
    except:
           vote_average.append('na')
            
    try:
           poster_path.append("https://image.tmdb.org/t/p/w500" + response_json['results'][0]['poster_path'])
    except:
           poster_path.append('na')

In [4]:
tmdb_data = pd.DataFrame({
    'title':movie_name,
    'original_title':original_title,
    'popularity':popularity,
    'vote_count':vote_count,
    'vote_average':vote_average,
    'poster_path':poster_path
})

In [5]:
# double check data
tmdb_data.head()

,title,original_title,popularity,vote_count,vote_average,poster_path
0,na,na,na,na,na,na
1,Manhattan Romance,Manhattan Romance,3.919,24,4.8,na
2,Stonehearst Asylum,Stonehearst Asylum,20.826,817,6.7,https://image.tmdb.org/t/p/w500/fZxGCCQ0NAtrae...
3,The Runner,The Runner,9.011,104,4.8,https://image.tmdb.org/t/p/w500/uIxFfrGlKEHJgk...
4,6 Years,6 Years,6.515,594,5.5,na


In [6]:
# remove any results where the title was na
tmdb_data_clean = tmdb_data
tmdb_data_clean = tmdb_data_clean.loc[tmdb_data_clean["title"]!= 'na']
tmdb_data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1282 entries, 1 to 1320
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           1282 non-null   object
 1   original_title  1282 non-null   object
 2   popularity      1282 non-null   object
 3   vote_count      1282 non-null   object
 4   vote_average    1282 non-null   object
 5   poster_path     1282 non-null   object
dtypes: object(6)
memory usage: 70.1+ KB


In [7]:
# save to csv
tmdb_data_clean.to_csv("tmdb_data_clean.csv", index=True)